# YouTube Crawling 
- Selenium을 사용한 이유는 스크롤을 내리지 않고 그냥 코드를 뽑으면 데이터의 양이 너무 적기 때문입니다. (30개 정도)

In [186]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max.colwidth',50)
pd.options.display.max_columns = 100

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [63]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from bs4 import BeautifulSoup
import re

In [83]:
driver = webdriver.Chrome('chromedriver.exe')
driver.maximize_window()

# YouTuber 1 - 승우아빠

### 영상  정보 수집 ( 제목 / url * / 업로드 날짜 / 조회수 / 좋아요 / 싫어요 )
- 전체 영상 목록이 보이는 페이지에서 각 동영상의 url 추출하는 과정

In [189]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('chromedriver.exe')
driver.get('https://www.youtube.com/channel/UCyozK5OFN5lDrwim5wqQnLA/videos')
body = driver.find_element_by_tag_name('body')
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 
endk = 5 # 5번 내림
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

SW_url = []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    SW_url.append(url)

- 추출한 모든 영상의 url을 입력하여, 각 페이지 안으로 들어가 정보를 수집

In [190]:
SW = pd.DataFrame({'youtuber' :[],
                   'subscribers' : [],
                   'name':[],
#                    'video_url':[],
                   'upload_date':[],
                   'hits' :[],
                   'likes_num':[],
                   'dislikes_num':[]})

for i in range(0,len(SW_url)):
    
    youtuber = soup.find('title').text
    sub_num = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
    name = video_list[i].find('a',{'id':'video-title'}).text
#     url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    

    # 각 동영상 페이지 들어가기 시작 
    start_url = SW_url[i]
    driver.get(start_url) 
    body = driver.find_element_by_tag_name('body')
    
    time.sleep(1.5)
    
    # 댓글까지 쭉 보려면 스크롤
    endk = 5 # 5번 내림
    while endk:
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        time.sleep(0.3) # 내리는 시간 간격 
        endk -= 1    
    page = driver.page_source 
    html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로
    
    
    # 각 동영상 페이지 들어가서 얻을 수 있는 정보
    pre_upload_date = html.find('div',{'id':'date'})
    upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text   
    pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
    hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
    likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
    dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'

    # 마지막에 모은 정보 합치기
    insert_data = pd.DataFrame({'youtuber' :[youtuber],
                                'subscribers' : [sub_num],
                                'name':[name],
#                                 'video_url':[url],
                                'upload_date':[upload_date], 
                                'hits' : [hits],
                                'likes_num':[likes_num],
                                'dislikes_num':[dislikes_num]})
    
    SW = SW.append(insert_data)

In [191]:
SW

,youtuber,subscribers,name,upload_date,hits,likes_num,dislikes_num
0,취미로 요리하는 남자 Yonam - YouTube,구독자 55.6만명,요리사들이 한 자리에 모이면 벌어지는 일 2 (오리다리콩피),최초 공개: 2020. 3. 25.,"238,800회",5.7천개,69개
0,취미로 요리하는 남자 Yonam - YouTube,구독자 55.6만명,sub)요리하는 남자의 간단한 집들이음식,최초 공개: 2020. 3. 4.,"629,179회",9.5천개,124개
0,취미로 요리하는 남자 Yonam - YouTube,구독자 55.6만명,(종료)제가 여러분께 요리를 해드립니다.,2020. 2. 20.,"160,897회",3.4천개,21개
0,취미로 요리하는 남자 Yonam - YouTube,구독자 55.6만명,sub)요리사들이 한 자리에 모이면 벌어지는 일.,최초 공개: 2020. 2. 4.,"1,961,937회",2.1만개,384개
0,취미로 요리하는 남자 Yonam - YouTube,구독자 55.6만명,sub)요리하는 남자들의 흔한 파티음식.,최초 공개: 2020. 1. 23.,"719,942회",9천개,143개
0,취미로 요리하는 남자 Yonam - YouTube,구독자 55.6만명,"ENG)요리하는 남자의 글램핑, 스테이크, 비프립, 돌문어, 또 라면...ㅋ",최초 공개: 2019. 12. 5.,"477,152회",5.4천개,74개
0,취미로 요리하는 남자 Yonam - YouTube,구독자 55.6만명,취미로 요리하는 남자가 무려 다섯명. feat. 초밥도전,최초 공개: 2019. 11. 26.,"696,162회",6.8천개,129개
0,취미로 요리하는 남자 Yonam - YouTube,구독자 55.6만명,SUB) 바닷가에서 먹는 브런치...는 개뿔.,최초 공개: 2019. 11. 14.,"411,846회",5천개,82개
0,취미로 요리하는 남자 Yonam - YouTube,구독자 55.6만명,SUB) 요리하는 남자가 캠핑카를 타면 벌어지는 일.,최초 공개: 2019. 10. 29.,"556,068회",7천개,111개
0,취미로 요리하는 남자 Yonam - YouTube,구독자 55.6만명,SUB) 요리하는 남자. 캠핑카. 그리고 랍스터.,최초 공개: 2019. 10. 20.,"808,428회",9.5천개,211개
